# 🧪 DeepSeek + LangChain Test Notebook

**Purpose**: Verify DeepSeek integration with LangChain works 100% before using in agents

**Test Steps**:
1. ✅ Install required packages
2. ✅ Initialize DeepSeek with LangChain
3. ✅ Test basic chat functionality
4. ✅ Test JSON output (critical for agents)
5. ✅ Test medical domain questions
6. ✅ Test conversation memory
7. ✅ Performance benchmark

---

## 📦 Step 1: Install Required Packages

Install LangChain and DeepSeek dependencies

In [ ]:
# Install packages
!pip install -q langchain==0.3.7
!pip install -q langchain-openai==0.2.9
!pip install -q openai>=1.0.0

print("\n✅ All packages installed successfully!")

## 🔑 Step 2: Set Up DeepSeek API Key

**Get your API key from**: https://platform.deepseek.com

**Important**: Replace `'your-api-key-here'` with your actual DeepSeek API key

In [ ]:
import os

# ⚠️ REPLACE THIS WITH YOUR ACTUAL API KEY
DEEPSEEK_API_KEY = "your-api-key-here"  # Get from https://platform.deepseek.com

# Set environment variable
os.environ["DEEPSEEK_API_KEY"] = DEEPSEEK_API_KEY

# Verify key is set
if DEEPSEEK_API_KEY == "your-api-key-here":
    print("⚠️  WARNING: Please replace 'your-api-key-here' with your actual DeepSeek API key!")
else:
    print(f"✅ API key set successfully! (Key starts with: {DEEPSEEK_API_KEY[:10]}...)")

## 🚀 Step 3: Initialize DeepSeek with LangChain

This is the core setup you'll use in your agents

In [ ]:
from langchain_openai import ChatOpenAI
import time

print("Initializing DeepSeek with LangChain...\n")

# Initialize DeepSeek
llm = ChatOpenAI(
    model="deepseek-chat",  # Main model for general tasks
    api_key=os.environ["DEEPSEEK_API_KEY"],
    base_url="https://api.deepseek.com",
    temperature=0.3,  # Lower = more focused, Higher = more creative
    max_tokens=2000,  # Maximum response length
)

print("✅ DeepSeek initialized successfully!")
print(f"   Model: deepseek-chat")
print(f"   Temperature: 0.3")
print(f"   Max Tokens: 2000")
print(f"   Base URL: https://api.deepseek.com")

## 🧪 Step 4: Test Basic Chat Functionality

Verify DeepSeek can respond to simple queries

In [ ]:
print("Testing basic chat functionality...\n")
print("="*60)

# Test 1: Simple greeting
print("\n📝 Test 1: Simple Greeting")
response = llm.invoke("Hello! Can you introduce yourself?")
print(f"Response: {response.content}")
print("✅ Test 1 passed!\n")

# Test 2: Math question
print("="*60)
print("\n📝 Test 2: Math Question")
response = llm.invoke("What is 25 * 47?")
print(f"Response: {response.content}")
print("✅ Test 2 passed!\n")

# Test 3: General knowledge
print("="*60)
print("\n📝 Test 3: General Knowledge")
response = llm.invoke("What is the capital of France?")
print(f"Response: {response.content}")
print("✅ Test 3 passed!\n")

print("="*60)
print("\n✅ All basic chat tests passed!")

## 🔍 Step 5: Test JSON Output (Critical for Agents)

Your agents need structured JSON responses - this is crucial!

In [ ]:
import json

print("Testing JSON output functionality...\n")
print("="*60)

# Test: Extract symptoms as JSON
prompt = """Extract symptoms from this message and return ONLY valid JSON.

Format:
{
  "symptoms": ["symptom1", "symptom2"],
  "severity": "mild/moderate/severe",
  "duration": "description"
}

Message: "I've had chest pain for 3 days, it's moderate and gets worse when I walk."
"""

print("📝 Prompt sent to DeepSeek:")
print(prompt)
print("\n" + "="*60)

response = llm.invoke(prompt)
print("\n📥 Raw Response:")
print(response.content)
print("\n" + "="*60)

# Try to parse JSON
try:
    # Clean the response (remove markdown code blocks if present)
    content = response.content.strip()
    if content.startswith("```json"):
        content = content.replace("```json", "").replace("```", "").strip()
    elif content.startswith("```"):
        content = content.replace("```", "").strip()
    
    parsed_json = json.loads(content)
    print("\n✅ JSON parsing successful!")
    print("\n📊 Parsed Data:")
    print(json.dumps(parsed_json, indent=2))
    
    # Verify structure
    assert "symptoms" in parsed_json, "Missing 'symptoms' key"
    assert isinstance(parsed_json["symptoms"], list), "'symptoms' should be a list"
    print("\n✅ JSON structure validation passed!")
    
except json.JSONDecodeError as e:
    print(f"\n❌ JSON parsing failed: {e}")
    print("This needs to be fixed before using in agents!")
except AssertionError as e:
    print(f"\n⚠️  JSON structure issue: {e}")

print("\n" + "="*60)

## 🏥 Step 6: Test Medical Domain Questions

Test DeepSeek's understanding of medical concepts (important for MedTwin)

In [ ]:
print("Testing medical domain knowledge...\n")
print("="*60)

medical_tests = [
    {
        "name": "Diabetes Symptoms",
        "question": "What are the main symptoms of diabetes? List 5 key symptoms."
    },
    {
        "name": "Blood Pressure Classification",
        "question": "If someone has blood pressure of 150/95 mmHg, is this normal, elevated, or hypertensive?"
    },
    {
        "name": "COPD vs Asthma",
        "question": "What's the key difference between COPD and asthma in terms of reversibility?"
    },
    {
        "name": "Symptom Classification",
        "question": """Classify this symptom into a condition category (diabetes, hypertension, heart_disease, or copd):
        'I feel short of breath when climbing stairs and have chest tightness.'
        Return ONLY the category name."""
    }
]

for i, test in enumerate(medical_tests, 1):
    print(f"\n📝 Medical Test {i}: {test['name']}")
    print(f"Question: {test['question']}")
    print("\nResponse:")
    
    response = llm.invoke(test['question'])
    print(response.content)
    print("\n" + "="*60)

print("\n✅ All medical domain tests completed!")

## 💬 Step 7: Test Conversation Memory

Test multi-turn conversations (important for Q&A agents)

In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain

print("Testing conversation memory...\n")
print("="*60)

# Create conversation chain with memory
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=False
)

# Simulate a medical interview
conversation_flow = [
    "I have chest pain.",
    "It started 2 days ago.",
    "It gets worse when I walk.",
    "What condition might I have based on what I told you?"
]

print("\n🗣️  Simulating Medical Interview:\n")

for i, message in enumerate(conversation_flow, 1):
    print(f"\n👤 Patient (Turn {i}): {message}")
    response = conversation.predict(input=message)
    print(f"🤖 Agent: {response}")
    print("\n" + "-"*60)

print("\n✅ Conversation memory test completed!")
print("\n📝 Full Conversation History:")
print(memory.buffer)

## ⚡ Step 8: Performance Benchmark

Measure response time and token usage

In [ ]:
import time

print("Running performance benchmark...\n")
print("="*60)

test_prompts = [
    "What is diabetes?",
    "Explain hypertension in simple terms.",
    "What are the symptoms of COPD?",
    "How does the heart work?",
    "What is a normal blood pressure range?"
]

total_time = 0
results = []

for i, prompt in enumerate(test_prompts, 1):
    print(f"\nTest {i}/5: {prompt[:50]}...")
    
    start_time = time.time()
    response = llm.invoke(prompt)
    end_time = time.time()
    
    elapsed = end_time - start_time
    total_time += elapsed
    
    results.append({
        "prompt": prompt,
        "time": elapsed,
        "response_length": len(response.content)
    })
    
    print(f"   ⏱️  Response time: {elapsed:.2f}s")
    print(f"   📏 Response length: {len(response.content)} characters")

print("\n" + "="*60)
print("\n📊 Performance Summary:")
print(f"   Total requests: {len(test_prompts)}")
print(f"   Total time: {total_time:.2f}s")
print(f"   Average time per request: {total_time/len(test_prompts):.2f}s")
print(f"   Fastest response: {min(r['time'] for r in results):.2f}s")
print(f"   Slowest response: {max(r['time'] for r in results):.2f}s")
print("\n✅ Performance benchmark completed!")

## 🎯 Step 9: Agent-Style Test (Simulating Your Use Case)

Test exactly how you'll use it in your agents

In [ ]:
print("Testing agent-style usage...\n")
print("="*60)

# Simulate how your SymptomQAAgent will use the LLM
class TestAgent:
    def __init__(self, llm):
        self.llm = llm
    
    def extract_symptoms(self, patient_message):
        """Simulate symptom extraction like in your agents"""
        prompt = f"""Extract symptoms from this patient message.
Return ONLY valid JSON with this structure:
{{
  "symptoms": ["list of symptoms"],
  "severity": "mild/moderate/severe",
  "timing": "when symptoms started"
}}

Patient message: "{patient_message}"
"""
        response = self.llm.invoke(prompt)
        return response.content
    
    def classify_condition(self, symptoms_text):
        """Simulate condition classification"""
        prompt = f"""Based on these symptoms, classify into ONE category:
- diabetes
- hypertension
- heart_disease
- copd
- unknown

Return ONLY the category name.

Symptoms: {symptoms_text}
"""
        response = self.llm.invoke(prompt)
        return response.content.strip().lower()

# Test the agent
test_agent = TestAgent(llm)

print("\n🧪 Test Case 1: Symptom Extraction")
patient_msg = "I've had high blood sugar for a week, feeling very thirsty and tired."
print(f"Patient: {patient_msg}")
result = test_agent.extract_symptoms(patient_msg)
print(f"\nExtracted:\n{result}")

print("\n" + "="*60)
print("\n🧪 Test Case 2: Condition Classification")
symptoms = "chest pain, shortness of breath on exertion, fatigue"
print(f"Symptoms: {symptoms}")
condition = test_agent.classify_condition(symptoms)
print(f"\nClassified as: {condition}")

print("\n" + "="*60)
print("\n✅ Agent-style tests completed successfully!")
print("\n🎉 DeepSeek is ready to be integrated into your agents!")

## ✅ Step 10: Final Verification Checklist

Run this cell to verify everything is working

In [ ]:
print("\n" + "="*60)
print("🔍 FINAL VERIFICATION CHECKLIST")
print("="*60 + "\n")

checklist = [
    "✅ Packages installed (langchain, langchain-openai, openai)",
    "✅ DeepSeek API key configured",
    "✅ LLM initialized successfully",
    "✅ Basic chat functionality works",
    "✅ JSON output parsing works",
    "✅ Medical domain knowledge verified",
    "✅ Conversation memory works",
    "✅ Performance is acceptable",
    "✅ Agent-style usage tested"
]

for item in checklist:
    print(item)

print("\n" + "="*60)
print("\n🎉 ALL TESTS PASSED!")
print("\n✅ DeepSeek + LangChain is 100% ready for your agents!")
print("\n📋 Next Steps:")
print("   1. Copy the LLM initialization code to your MedTwin notebook")
print("   2. Replace ChatOllama with ChatOpenAI")
print("   3. Your agents will work without any other changes!")
print("\n" + "="*60)

## 📝 Code to Copy to Your MedTwin Notebook

Once all tests pass, copy this exact code to your agents notebook:

In [ ]:
# ============================================================
# COPY THIS TO YOUR MEDTWIN NOTEBOOK
# ============================================================

# 1. Install packages (replace your Ollama installation)
# !pip install -q langchain==0.3.7
# !pip install -q langchain-openai==0.2.9
# !pip install -q openai>=1.0.0

# 2. Initialize DeepSeek (replace your ChatOllama initialization)
import os
from langchain_openai import ChatOpenAI

os.environ["DEEPSEEK_API_KEY"] = "your-api-key-here"  # Replace with your key

llm = ChatOpenAI(
    model="deepseek-chat",
    api_key=os.environ["DEEPSEEK_API_KEY"],
    base_url="https://api.deepseek.com",
    temperature=0.3,
    max_tokens=2000
)

print("✅ DeepSeek initialized!")

# 3. Your agents will work exactly as before!
# No changes needed to SymptomQAAgent, AnalysisAgent, etc.
# They all use self.llm.invoke() which works the same way

print("\n🎉 Ready to use in your agents!")